### Overview

This short lesson will demonstrate how to take a postcode or ZIP code - for example, entered by a user from a web form - and do a lookup on that postcode to get more detailed geographical data.

This data can be used to, for example, show a map of where your users are located - by transforming postcodes into `latitude` and `longitude`, finding bounding boxes for the postcode boundary, etc.

We're going to use the Nominatim OpenStreetMap API to do the lookup. Specifically, we're going to use the search functionality that the API provides - more info on this API [can be found here](https://nominatim.org/release-docs/latest/api/Search/).

To make HTTP requests to this API, we'll use the `requests` package.

In [ ]:
-m pip install ipykernel -U --user --force-reinstall

In [1]:
import requests
from pprint import pprint

Let's define the base URL for the Nominatim API. We specify `format=json` because we want the output in JSON - however, other types are available (for example, XML and GeoJSON).

In [2]:
BASE_URL = 'https://nominatim.openstreetmap.org/search?format=json'

Let's perform a GET request to this endpoint, and pass a postcode and a zipcode to retrieve the geographical information.

In [3]:
postcode = 'G42 9AY'

response = requests.get(f"{BASE_URL}&postalcode={postcode}")
data = response.json()
pprint(data)

[{'addresstype': 'postcode',
  'boundingbox': ['55.6652300', '55.9852300', '-4.4132000', '-4.0932000'],
  'class': 'place',
  'display_name': 'Glasgow City, Alba / Scotland, G42 9AY, United Kingdom',
  'importance': 0.12500009999999995,
  'lat': '55.82523',
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. '
             'http://osm.org/copyright',
  'lon': '-4.2532',
  'name': 'G42 9AY',
  'place_id': 329404231,
  'place_rank': 25,
  'type': 'postcode'}]


Let's extract the bounding box and the latitude/longitude.

In [4]:
latitude = data[0].get('lat')
longitude = data[0].get('lon')
print(latitude, longitude)

bbox = data[0].get('boundingbox')
print(bbox)

55.82523 -4.2532
['55.6652300', '55.9852300', '-4.4132000', '-4.0932000']


Now, let's look for a generic ZIP code - `2601`.



In [5]:
zipcode = '2601'

response = requests.get(f"{BASE_URL}&postalcode={zipcode}")
response.json()

[{'place_id': 368562940,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'lat': '-33.64315427173913',
  'lon': '-61.85790109642857',
  'class': 'place',
  'type': 'postcode',
  'place_rank': 21,
  'importance': 0.22500009999999993,
  'addresstype': 'postcode',
  'name': '2601',
  'display_name': 'Municipio de Murphy, 2601, Departamento General López, Santa Fe, Argentina',
  'boundingbox': ['-33.8031543', '-33.4831543', '-62.0179011', '-61.6979011']},
 {'place_id': 330908002,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
  'lat': '16.454289474175823',
  'lon': '120.59515791593407',
  'class': 'place',
  'type': 'postcode',
  'place_rank': 21,
  'importance': 0.22500009999999993,
  'addresstype': 'postcode',
  'name': '2601',
  'display_name': 'La Trinidad, 2601, Benguet, Cordillera Administrative Region, Philippines',
  'boundingbox': ['16.2942895', '16.6142895', '120.4351579', '120.7551579']},
 {'place_id':

We can see that there's a lot of results - many different countries have a ZIP code of 2601, including Austria, Australia, Hungary and Italy.

We can append another query parameter to narrow this down to one country - let's say Italy. Just add `&country=italia` to the URL!

In [6]:
response = requests.get(f"{BASE_URL}&postalcode={zipcode}&country=italia")
data = response.json()

Now, we get a list with a single record, for the Italian location with ZIP code 2106.

To extract latitude and longitude, we can use the following code.

In [9]:
if data:
    latitude2 = data[0].get('lat')
    longitude2 = data[0].get('lon')
    print(latitude2, longitude2)
else:
    print("The data list is empty.")


The data list is empty.


In [7]:
latitude2 = data[0].get('lat')
longitude2 = data[0].get('lon')

print(latitude2, longitude2)

IndexError: list index out of range

### Mapping the Points

Now we'll look quickly at how to plot these points on a map. We'll use Python's `folium` library to do this - it is an interface into the `Leaflet` JavaScript mapping library.

In [18]:
latitude = "9.928310582743"
longitude = "78.12281896026138"

In [19]:
latitude2 = "13.085505095748271"
longitude2 = "80.27132898126216"

In [23]:
import folium

# create tuples representing our location
location = float(latitude), float(longitude)
location2 = float(latitude2), float(longitude2)

# center the map at Amsterdam
amsterdam = (18.651923658538383, 72.9150551991513)

# create a Folium map centred at the above location
m = folium.Map(location=amsterdam, zoom_start=4, width=800, height=400)

# add markers at the locations
folium.Marker(location, popup="Madurai").add_to(m)
folium.Marker(location2, popup="Chennai").add_to(m)

m

In [26]:
# generator expression to compute midpoint of the two locations
# this works because both locations are of form: (lat, long)
# zipping them together allows us to iterate over both lats at once,
# and then both lons at once
midpoint_gen = ((x+y)/2 for x,y in zip(location, location2))

# convert generator to a tuple representing lat/longitude of the midpoint
midpoint = tuple(midpoint_gen)

print(location)
print(location2)
print(midpoint)

(9.928310582743, 78.12281896026138)
(13.085505095748271, 80.27132898126216)
(11.506907839245635, 79.19707397076178)


Now, let's centre map at midpoint we found above, and display all the markers for each position.

In [ ]:
import folium

# create Folium map
m = folium.Map(location=midpoint, zoom_start=4, width=800, height=400)

# add marker at the locations
folium.Marker(location, popup="The postcode brought me here").add_to(m)
folium.Marker(location2, popup="The postcode brought me here").add_to(m)
folium.Marker(midpoint, popup="Middle!").add_to(m)

m

### Find the distances between the two points

We'll use `geopy` to find the geodesic distances between the points.

In [24]:
from geopy.distance import distance

km = distance(location, location2)
miles = distance(location, location2).miles

print("Distance between postcodes:")
print(f"{km}")
print(f"{miles} miles")

Distance between postcodes:
420.59151347147525 km
261.3434501706753 miles


### Folium bonus - Lines between points.

A small but useful feature of Folium is the ability to draw lines between points, using `folium.PolyLine()`. This is shown below, where we connect a line between the two locations.

In [34]:
import folium

# create a Folium map centred at the above location
m = folium.Map(location=midpoint, zoom_start=4, width=800, height=400)

# add marker at the locations
folium.Marker(location, popup="Madurai").add_to(m)
folium.Marker(location2, popup="Chennai").add_to(m)

# add line between points
folium.PolyLine((location,location2)).add_to(m)
# Add popup with distance information
folium.Marker(midpoint,popup="{km}").add_to(m)


In [32]:
import folium

# create a Folium map centred at the above location
m = folium.Map(location=midpoint, zoom_start=4, width=800, height=400)

# add marker at the locations
folium.Marker(location, popup="Madurai").add_to(m)
folium.Marker(location2, popup="Chennai").add_to(m)

# add line between points
folium.PolyLine((location,location2)).add_to(m)

m